## Réponses au test de deep learning 

Dans ce document je me suis plus attardé sur la éthode de résolution du problème en présentant les grandes étapes dans la démarche de résolution de chaque problème posé

#### 1 - Description de la mise en place d'un pipeline de traitement du process de détection d'un langage abusif sur le social web : 
Pour mettre sur pied ce pipeline il est question d’extraire un réseau conversationnel à partir de l’extraction des conversations récupérées, de ressortir les features et de les entrainer dans un classificateur. La définition des features peut imposer une analyse exploratoire en tenant compte du jargon associé  la population et au contexte. Les données recoltées peuvent être de sources diverses dans les lignes qui suivnet je vais présenter grossirement la méthode de récupération des messages contenus dans les tweets

In [ ]:
# récupération des données de tweet 

# authentification à l'api 
# modules nécessaires
import tweepy
import csv
import pandas 
import re
# Authenticate to Twitter
auth = tweepy.OAuthHandler("clé", "cle_secret")
auth.set_access_token("token_d'acces", "token_secret")
api = tweepy.API(auth)
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Erreur d'authentification")

with open('tendance.csv','wb') as file :
    colonne = ['user_name','tweet']
    writer = csv.DictWriter(file, fieldnames=colonne)
# Par exemple nous allons récupérer les derniers tweets en francais comprenant le mot greve
    for tweet in api.search(q="greve", lang="fr", rpp=100000000):
        writer.writerow('user_name': tweet.user.name,'tweet':tweet.text)

# Nettoyage des données 
tendance  = pd.read_csv('tendance.csv', delimiter=';')

def  clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df

tendance = clean_text(tendance,'tweet')

A ce niveau afin de plus se concentrer sur la méthode nous allons supposer qu'un traitement manuel des données récoltées nous a permis d'avoir une case décision pour marqué si le tweet est considéré comme abusif ou pas.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),
])
model = pipeline_sgd.fit(X_train, y_train)

On suppose que nous disposons d'un modèle obtenu sur la base du pipeline précédent. Nous alons juste prédire la classe de chaque tweet.

In [ ]:
y_predict = model.predict(X_test)

#### 2 - Utilisation d'un algorithme Hybrid CNN-LSTM pour la détection d'un langage abusif sur le social web

In [ ]:
import pyfacebook 
import pymongo

api = pyfacebook.InstagramApi(app_id='your app id', app_secret='your app secret',short_token='your short token')
content = api.get_medias(username='jacques_chirac')

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["data"]
x = mycol.insert_one(content)


In [ ]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# execution 
result = model.predict(content)